In [30]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 3

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import dotenv, os, shutil
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from utils import print_sample_node, setup_indexes, read_indexes, save_indexes
from utils import add_to_indexes, VECTOR_INDEX_PATH, SUMMARY_INDEX_PATH

Globals

In [ ]:

VERBOSE=True
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embedding = OpenAIEmbedding(model="text-embedding-ada-002")

In [ ]:
#pip install newspaper3k
#pip install lxml[html_clean]
#pip install llama-index


#### Create Llamaindex documents

Set up indexes for the first time

In [ ]:
for d in [VECTOR_INDEX_PATH, SUMMARY_INDEX_PATH]:
    if os.path.exists(d):
        shutil.rmtree(d)
indexes=vector_index, summary_index=setup_indexes()

Read articles from the web and add to index

In [ ]:
from read_news import get_stock_news
symbol="INTC"
articles=get_stock_news(symbol)
vector_index, summary_index = read_indexes()
add_to_indexes(articles, vector_index, summary_index)
save_indexes(vector_index, summary_index, dir_suffix="")

In [ ]:
#print_sample_node(VECTOR_INDEX_PATH)

In [ ]:
from typing import List
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.tools import QueryEngineTool, FunctionTool
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.vector_stores import FilterCondition, MetadataFilters

def calculator(a:float, b:float, op:str):
    """ performs operation op on a and b.  
    Possible 'ops'are as follows: 
    '+':addition
    '-': minus
    '*': multiply
    '/': divide
    '%chg': percentage change from a to b
    'chg':  absolute value of change
    """
    if op == "+": return a+b
    elif op == "-": return a-b
    elif op == "*": return a*b
    elif op == "/": return a/b
    elif op == "ch": return abs(a-b)
    elif op=="%chg": return round(100*(b/a-1))
    else:
         raise NotImplementedError

vector_query_engine=vector_index.as_query_engine(filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]))
summary_query_engine=summary_index.as_query_engine()

summary_tool = QueryEngineTool.from_defaults(
        name="summary-tool",
        query_engine=summary_query_engine,
        description=(
            "Useful for summarization questions related to stocks."
        ),
    )

vector_tool = QueryEngineTool.from_defaults(
        name="vector-tool",
        query_engine=vector_query_engine,
        description=(
            "Useful for retrieving specific context from stock news."
        )
    )

def vector_query(
    query: str, 
    publication_date: str=None
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    publication_date is the date on which the article was published.
    
    """

    metadata_dicts = []
    if publication_date is not None:
        metadata_dicts.append({"key": "published_at", "value": publication_date})
    
    print(metadata_dicts)
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

vector_query_tool = FunctionTool.from_defaults(
    name="vector_query_tool", 
    description="produces answers but limits to specified publication date",
    fn=vector_query
)    


def summary_query(
    query: str, 
    publication_date: str=None
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    publication_date is the date on which the article was published.
    
    """

    metadata_dicts = []
    if publication_date is not None:
        metadata_dicts.append({"key": "published_at", "value": publication_date})
    
    query_engine = summary_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

summary_query_tool = FunctionTool.from_defaults(
    name="summary_query_tool",
    fn=vector_query,    
    description="produces summary but limits to specified publication date"
)    

math_tool = FunctionTool.from_defaults(
    fn=calculator,
    name="math_tool",
    description="""Performs various math calculations: + - * / percent change & change\n
    "performs operation op on a and b.  
    Possible 'ops'are as follows: 
    '+':addition
    '-': minus
    '*': multiply
    '/': divide
    '%chg': percentage change from a to b
    'chg':  absolute value of change"""
)
# router=RouterQueryEngine.from_defaults(
#         selector=LLMSingleSelector.from_defaults(),
#         query_engine_tools=[summary_tool, vector_tool],
#         verbose=VERBOSE
#         )

# response=router.query(" what was the percentage change in Apple's earnings this quarter compared with the same quarter in the prior year?")
# print(str(response))

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [summary_query_tool,math_tool, vector_query_tool], 
    verbose=True
)
agent = AgentRunner(agent_worker)
response=agent.query("what was Intel's earnings this quarter  as reported on August 3 2024?")
print(str(response))

In [ ]:
agent.query("what is the change (in %) from 105 to 119?")

In [ ]:
agent.query("Summarize Apple news in bullet form on Aug 3, 2024")